In [ ]:
%run pylib/compare_accuracy
names=('Neural Net',"RBF SVM","Random Forest")

classifiers=(MLPClassifier(alpha=1, max_iter=1000),SVC(gamma=2, C=1,probability=True), RandomForestClassifier(n_estimators=100, max_features=2))

test = Compare_Classifiers(fs_data.getXy(specs1), unid.getXy(specs2),'files/test',names=names, classifiers=classifiers)


#Takes a tuple of classifiers and a tuple of their names
def pltAvgPrecRec(classifiers, names, X, y, Axis=None):

    import seaborn as sns
    import sklearn
    from sklearn.metrics import PrecisionRecallDisplay
    import matplotlib.pyplot as plt

    def getPrecRec(theX, they, clf, ax=None):
        X_train, X_test, y_train, y_test = train_test_split(theX, they, test_size=0.333)

        classifier = clf.fit(X_train, y_train)

        #Get precision and recall values
        tem = PrecisionRecallDisplay.from_estimator(
            classifier, X_test, y_test, name=name, ax=ax, plot_chance_level=True
        )
        tem.figure_.clear()

        return tem

    #set up
    thePrec = np.empty(0)
    theRecall = np.empty(0)
    theSet = np.empty(0)
    
    
    #Loop through classifiers
    for name, clf in zip(names, classifiers):
    
        they = (y=='psr')

        pr = getPrecRec(X, they, clf, Axis)

        count = 0
        prec = pr.precision
        recall = pr.recall


        while((count:=count+1) <= 20):

            pr = getPrecRec(X, they, clf, Axis)

            if prec.size < pr.precision.size:
                prec += pr.precision[:prec.size]
                recall += pr.recall[:recall.size]
            else:
                p = np.ones(prec.size)
                r = np.ones(recall.size)

                p = prec/count
                r = recall/count

                p[:pr.precision.size] = pr.precision
                r[:pr.recall.size] = pr.recall
                prec += p
                recall += r



        theSet = np.concatenate((theSet, np.full((prec.size), name)))
        thePrec = np.concatenate((thePrec, prec/count))
        theRecall = np.concatenate((theRecall, recall/count))

    prdf = pd.DataFrame(data={"prec": thePrec, 
                              "recall": theRecall, 
                              "group": theSet})

    plot = sns.lineplot(data=prdf, x="recall", y="prec", hue='group')
    
    return plot
    

In [ ]:
a = (SVC(gamma=2, C=1),RandomForestClassifier(n_estimators=100, max_features=2),MLPClassifier(alpha=1, max_iter=1000),)
b = ("svc", "RF", "Neural",)
t = pltAvgPrecRec(a, b, test.X, test.y,)

t.figure